In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from data import *
from features import *
from preprocess import *
from stats import *
from model import *
from step import *
import mlflow
import lightning as L
import logging
import math
import optuna
from functools import partial
import sklearn.linear_model, sklearn.neighbors
from lightning.pytorch.callbacks import ModelCheckpoint
import tempfile
from pathlib import Path
import torch

In [21]:
def prepare_data(data_func, monkey, area, subregion, phase, wins, num_samples, alpha, cell):
    # 0-1000 fixation, 1000-1500 cue, 1500-3000 delay, 3000-3500 sample, 3500-5000 delay2, 5000-6000 saccade
    # num_wins = math.floor((6000 - win_size - win_start) / win_stride) + 1

    step = Step(data_func)
    if monkey: step = Step(subset, step, key='monkey', value=monkey)
    if area: step = Step(subset, step, key='area', value=area)
    if subregion: step = Step(subset, step, key='subregion', value=subregion)
    if phase: step = Step(subset, step, key='phase', value=phase)
    if cell: step = Step(subset, step, key='cell', value=cell)
    step = Step(gt_n_trials, step, n=8)
    step = Step(win_cts_by_trial, step, wins=wins)
    step = Step(win_cts_by_win, step)
    step = Step(stat_tests, step)
    # topn_step = Step(top_n_by_cell_win_t, st_step, n=top_n)
    if alpha: 
        step = Step(sig_stat, step, stat='anova_p', alpha=alpha, keep_win=True)
        step = Step(select, step)
    step = Step(tvt_split, step, num_splits=10, num_val=1, num_test=1)
    step = Step(gen_samples, step, num_train=num_samples, num_val=1, num_test=1)
    return step()

In [22]:
def _diff_and_acc(yhat, y):
    diff = torch.abs(yhat - y)
    diff = torch.min(torch.stack([diff, 16 - diff]), axis=0)[0]
    # print(yhat, y, diff, torch.mean(diff))
    diff = torch.mean(diff)
    return diff, torch.mean((yhat == y).float())

In [23]:
y = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7], dtype=torch.float)
yhat = torch.tensor([7, 1, 3, 2, 4, 2, 1, 0], dtype=torch.float)
# diff = torch.abs(yhat - y)
# torch.min(torch.stack([diff, 8 - diff]), axis=0)[0]
torch.mean((y == yhat).float()), _diff_and_acc(yhat, y)

(tensor(0.2500), (tensor(3.), tensor(0.2500)))

In [24]:
def train_skl_model(model, data, tags, params, i, others):
    train = data['train'][i]
    val = data['val'][i]
    test = data['test'][i]
    
    X = train[:, :-1]
    m = torch.mean(X, dim=0)
    s = torch.std(X, dim=0)
    s[s == 0] = 1

    train_X, val_X, test_X = ((x[:, :-1] - m) / s for x in (train, val, test))
    train_y, val_y, test_y = (x[:, -1] for x in (train, val, test))

    # if model is KNeighborsClassifier, use k_neighbors from others as n_neighbors
    if model == sklearn.neighbors.KNeighborsClassifier:
        model.set_params(n_neighbors=others['k_neighbors'])

    model.fit(train_X, train_y)

    train_yhat = torch.tensor(model.predict(train_X))
    train_diff, train_acc = _diff_and_acc(train_yhat, train_y)
    # train_acc = model.score(train_X, train_y)
    val_yhat = torch.tensor(model.predict(val_X))
    val_diff, val_acc = _diff_and_acc(val_yhat, val_y)
    # val_acc = model.score(val_X, val_y)
    test_yhat = torch.tensor(model.predict(test_X))
    test_diff, test_acc = _diff_and_acc(test_yhat, test_y)
    # test_acc = model.score(test_X, test_y)
    
    return train_acc, val_acc, test_acc, model, (train_X, train_y), (val_X, val_y), (test_X, test_y), train_diff, val_diff, test_diff

In [25]:
def _fnn_diff(dataloader, model):
    model.eval()
    yhat = model.get_predicted_positions(model(dataloader.dataset.tensors[0]))
    y = dataloader.dataset.tensors[1][:,-1]
    # print(dataloader.dataset.tensors[0].shape, dataloader.dataset.tensors[1].shape, yhat.shape, y.shape)
    return _diff_and_acc(yhat, y)

In [26]:
def _train_pytl_model(model_class, data, tags, params, i, others):
    callback = ModelCheckpoint(save_top_k=1, monitor='val_acc', mode='max')
    # mlflow.pytorch.autolog(checkpoint_monitor='val_acc', checkpoint_mode='max')

    train, val, test = data['train'][i], data['val'][i], data['test'][i]
    data_module = DataModule(*[train, val, test], one_hot_max=others['one_hot_max'], train_batch_size=others['train_batch_size'])
    input_size = data_module.train_dataloader().dataset[0][0].shape[0]
    model = model_class(input_size, others['hidden_sizes'], 8)

    trainer = L.Trainer(max_epochs=20, enable_progress_bar=False, callbacks=[callback])
    trainer.fit(model, data_module)
    # train_acc = trainer.logged_metrics['best_train_acc']
    # val_acc = trainer.logged_metrics['best_val_acc']
    model = model_class.load_from_checkpoint(callback.best_model_path)
    test_results = trainer.test(model, data_module, verbose=False) #, ckpt_path="best")
    train_diff, train_acc = _fnn_diff(data_module.train_dataloader(), model)
    val_diff, val_acc = _fnn_diff(data_module.val_dataloader(), model)
    test_diff, test_acc = _fnn_diff(data_module.test_dataloader(), model)
    return (train_acc, val_acc, test_acc, model, 
    # return (train_acc, val_acc, test_results[0]['test_acc'], model, 
            data_module.train_dataloader().dataset, data_module.val_dataloader().dataset, data_module.test_dataloader().dataset,
            train_diff, val_diff, test_diff)

def train_pytl_model(model_class, data, tags, params, i, others):
    # with mlflow.start_run(nested=True):
    train_acc, val_acc, test_acc, model, train_data, val_data, test_data, train_diff, val_diff, test_diff = _train_pytl_model(model_class, data, tags, params, i, others)
        # mlflow.log_metrics({'train_acc': train_acc.item(), 'val_acc': val_acc.item(), 'test_acc': test_acc})
        # mlflow.set_tags(tags)
    return train_acc, val_acc, test_acc, model, train_data, val_data, test_data, train_diff, val_diff, test_diff

In [27]:
def skl_model_func(model, data, tags, params, others): 
    return Parallel(n_jobs=8)(
        delayed(train_skl_model)(model, data, tags, params, i, others) 
        for i in range(len(data['train'])))
    # return [train_skl_model(model, data, tags, params, i, others) for i in range(len(data['train']))]

def pytl_model_func(model_class, data, tags, params, others): 
    return [train_pytl_model(model_class, data, tags, params, i, others)
            for i in range(len(data['train']))]

In [28]:
def skl_trial(trial, win_options, model):
    others = {}
    num_train_samples = 120 #trial.suggest_int('num_train_samples', 50, 150, step=10)
    win_opt_strs = {str(w) : w for w in win_options}
    win_str = trial.suggest_categorical('wins', win_opt_strs)
    wins = win_opt_strs[win_str]
    # top_func_s = trial.suggest_categorical('top_func', ['cell_t', 'cell_win_t', 'cell_kruskal', 'cell_win_kruskal', 'cell_anova', 'cell_win_anova'])
    # top_func = {'cell_t': top_n_by_cell_t, 'cell_win_t': top_n_by_cell_win_t, 
    #             'cell_kruskal': top_n_by_cell_kruskal, 'cell_win_kruskal': top_n_by_cell_win_kruskal,
    #             'cell_anova': top_n_by_cell_anova, 'cell_win_anova': top_n_by_cell_win_anova
    #             }[top_func_s]
    top_func = 'NA'
    # top_n = trial.suggest_int('top_n', 14, 20)
    top_n = -1
    # win_size = trial.suggest_int('win_size', 250, 500, step=50)
    # win_stride = win_size
    if issubclass(type(model), sklearn.neighbors.KNeighborsClassifier):
        others['k_neighbors'] = trial.suggest_int('k_neighbors', 3, 15)

    # alpha = trial.suggest_categorical('alpha', [0.02, 0.03, 0.04])
    alpha = 0.1 # 10**trial.suggest_float('log10_alpha', -4.5, -1.5, step=1)
    return num_train_samples, top_func, top_n, wins, alpha, others

In [29]:
def pytl_trial(trial, win_start, model):
    num_train_samples, top_func, top_n, win_size, win_stride, others = skl_trial(trial, win_start, model)
    hidden_sizes = [2**trial.suggest_int('hidden_size_exp', 7, 9)] * trial.suggest_int('num_hidden_layers', 2, 3)
    one_hot_max = 0.9 #trial.suggest_float('one_hot_max', 0.7, 0.975, step=0.025)
    train_batch_size = 4 #trial.suggest_int('train_batch_size', 1, 10, step=3)
    return num_train_samples, top_func, top_n, win_size, win_stride, {'hidden_sizes': hidden_sizes, 'one_hot_max': one_hot_max, 'train_batch_size': train_batch_size}

In [30]:
def skl_log(models, vals, tests, i):
    model = mlflow.sklearn.log_model(models[i], 'model')
    dataset = mlflow.data.from_numpy(tests[i][0].numpy(), targets=tests[i][1].numpy())
    mlflow.log_input(dataset, context='test')
    dataset = mlflow.data.from_numpy(vals[i][0].numpy(), targets=vals[i][1].numpy())
    mlflow.log_input(dataset, context='val')

In [31]:
def pytl_log(models, vals, tests, i):
    model = mlflow.pytorch.log_model(models[i], 'model')
    dataset = mlflow.data.from_numpy(vals[i][:][0].numpy(), targets=vals[i][:][1].numpy())
    mlflow.log_input(dataset, context='val')
    dataset = mlflow.data.from_numpy(tests[i][:][0].numpy(), targets=tests[i][:][1].numpy())
    mlflow.log_input(dataset, context='test')

In [32]:
def objective(data_func, model_func, model, monkey, area, subregion, phase, win_options, tags, trial_func, log_func, cell, trial):
    num_train_samples, top_func, top_n, wins, alpha, others = trial_func(trial, win_options, model)

    trials_to_consider = trial.study.get_trials(deepcopy=False, states=(optuna.trial.TrialState.COMPLETE,))
    for t in reversed(trials_to_consider):
        if trial.params == t.params:
            # print('skip repeat')
            # return t.value
            print('repeat')

    try:
        data = prepare_data(data_func, monkey, area, subregion, phase, wins, num_train_samples, alpha, cell=cell)
        tags['n_cells'] = len(data['cells'])

        if cell is not None:
            tags['cell'] = cell

        with mlflow.start_run(nested=True):
            # mlflow.log_table(data['win_cts_by_win'].index, 'win_cts_by_win')
            # mlflow.data.from_pandas(pd.DataFrame(data['win_cts_by_win'].index))
            cellwindows = pd.DataFrame(data['win_cts_by_win'].index.droplevel(1).unique().to_series()).reset_index()[['cell', 'win']]

            with tempfile.TemporaryDirectory() as tmp_dir:
                path = Path(tmp_dir, "cellwindows.csv")
                cellwindows.to_csv(path, index=False)
                mlflow.log_artifact(path)
            
            # mlflow.log_input(mlflow.data.from_pandas(cellwindows, 'cellwindows'))
            mlflow.set_tags(tags)
            mlflow.log_params(trial.params)

            results = model_func(model, data, tags, trial.params, others)
            train_accs, val_accs, test_accs, models, trains, vals, tests, train_diff, val_diff, test_diff = zip(*results)
            std = np.std(val_accs)

            mlflow.log_metric('train_acc', np.mean(train_accs))
            mlflow.log_metric('val_acc', np.mean(val_accs))
            mlflow.log_metric('test_acc', np.mean(test_accs))
            mlflow.log_metric('train_diff', np.mean(train_diff))
            mlflow.log_metric('val_diff', np.mean(val_diff))
            mlflow.log_metric('test_diff', np.mean(test_diff))
            mlflow.log_metric('val_std', std)
            note = f'train_accs={np.stack(train_accs)}, val_accs={np.stack(val_accs)}, test_accs={np.stack(test_accs)}'
            mlflow.set_tag('mlflow.note.content', note)

            # i = np.argmax(val_accs)
            # log_func(models, vals, tests, i)

            # dataset = mlflow.data.from_numpy(trains[i][0].numpy(), targets=trains[i][1].numpy())
            # mlflow.log_input(dataset, context='train')
            # mlflow.evaluate(data=dataset, model_type='classifier', model=model.model_uri)

        return np.mean(val_accs)
    except Exception as e:
        # return 0
        raise e

In [33]:
def try_model(experiment_name, data_func, model, monkey, area, subregion, phase, win_options, n_trials, existing_mode='abort', cell=None):
    if isinstance(model, type) and issubclass(model, L.LightningModule):
        model_func, trial_func, log_func, model_name = (pytl_model_func, pytl_trial, pytl_log, model.__name__)
    else:
        model_func, trial_func, log_func, model_name = (skl_model_func, skl_trial, skl_log, type(model).__name__)

    criteria = []
    if monkey: criteria.append(f'tags.monkey = "{monkey}"')
    if area: criteria.append(f'tags.area = "{area}"')
    if subregion: criteria.append(f'tags.subregion = "{subregion}"')
    if phase: criteria.append(f'tags.phase = "{phase}"')
    if cell: criteria.append(f'tags.cell = "{cell}"')
    criteria.append(f'tags.model = "{model_name}"')

    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is not None:
        experiment_id = experiment.experiment_id
        runs = mlflow.search_runs(experiment_ids=experiment_id, filter_string=' and '.join(criteria))

        if not runs.empty:
            print(f'Experiment {experiment_name} already has runs with criteria {criteria}')
            if existing_mode == 'abort':
                print('Aborting')
                return
            elif existing_mode == 'overwrite':
                print('Deleting runs')
                for ind, run in runs.iterrows():
                    if run['status'] != 'RUNNING':
                        mlflow.delete_run(run["run_id"])
            elif existing_mode == 'append':
                print('Appending')

    mlflow.set_experiment(experiment_name)

    with mlflow.start_run():
        study = optuna.create_study(direction='maximize') #, sampler=optuna.samplers.BruteForceSampler())

        tags = {'model': model_name}
        if monkey: tags['monkey'] = monkey
        if area: tags['area'] = area
        if subregion: tags['subregion'] = subregion
        if phase: tags['phase'] = phase
        if cell: tags['cell'] = cell
        mlflow.set_tags(tags)

        study.optimize(partial(objective, data_func, model_func, model, monkey, area, subregion, phase, win_options, tags, trial_func, log_func, cell), n_trials=n_trials)

        print(study.best_params, study.best_value)

        mlflow.log_params(study.best_params)
        mlflow.log_metric('best_val_acc', study.best_value)

In [34]:
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
logging.getLogger("pytorch_lightning.utilities").setLevel(logging.ERROR)

In [35]:
# data_type = 'msng'
# area = None
# phase = None
# win_opt_1 = [[x, x + 125] for x in range(1500, 5000, 125)]
# win_opt_1 = [[1500, 2000], [2000, 2500], [2500, 3000], [3000, 3500], [3500, 4000], [4000, 4500], [4500, 5000]]
# exps = [
#     (None, 'PFC', None),
#     (None, 'PPC', None),
#     (None, 'PFC', '46'),
#     (None, 'PFC', '8'),
#     (None, 'PPC', '7a'),
#     (None, 'PPC', 'LIP'),
#     ('KEN', 'PFC', None),
#     ('LEM', 'PFC', None),
#     ('KEN', 'PPC', None),
#     ('LEM', 'PPC', None),
#     ('KEN', 'PPC', '7a'),
#     ('LEM', 'PPC', '7a'),
#     ('KEN', 'PPC', 'LIP'),
#     ('LEM', 'PPC', 'LIP'),
# ]

# data_type = 'mnm_feature'
# area = 'PFC'
# win_opt_1 = [[x, x + 125] for x in range(1500, 5000, 125)]
# win_opt_1 = [[1500, 2000], [2000, 2500], [2500, 3000], [3000, 3500], [3500, 4000], [4000, 4500], [4500, 5000]]
# exps = [
#     (None, None, "PRE"),
#     (None, None, "POST"),
#     (None, "MD", "PRE"),
#     (None, "MD", "POST"),
#     (None, "PV", "PRE"),
#     (None, "PV", "POST"),
#     (None, "AD", "PRE"),
#     (None, "PD", "POST"),
#     ('BEN', 'MD', 'PRE'),
#     ('ELV', 'MD', 'PRE'),
#     ('ELV', 'MD', 'POST'),
#     ('ELV', 'PV', 'PRE'),
#     ('ELV', 'PV', 'POST'),
#     ('MAN', 'AD', 'PRE')
# ]

data_type = 'mnm_spatial'
# area = 'PFC'
# win_opt_1 = [
#     [1500, 1750], [1750, 2000],
#     [2000, 2250], [2250, 2500],
#     [2500, 2750], [2750, 3000],
#     [3000, 3250], [3250, 3500],
#     [3500, 3750], [3750, 4000],
#     [4000, 4250], [4250, 4500],
#     [4500, 4750], [4750, 5000],
# ]
win_opt_1 = [[x, x + 125] for x in range(1500, 5000, 125)]
# exps = [
# (None, None, "PRE"),
# (None, None, "POST"),
# (None, "MD", "PRE"),
# (None, "MD", "POST"),
# (None, "PD", "PRE"),
# (None, "PD", "POST"),
# (None, "PV", "PRE"),
# (None, "PV", "POST"),
# (None, "AD", "POST"),
# ("ELV", "MD", "PRE"),
# ("ELV", "MD", "POST"),
# ("ELV", "PV", "POST"),
# ("ELV", "MD", "PRE"),
# ("ELV", "MD", "PRE"),
# ("ADR", "PD", "POST"),
# ("NIN", "AD", "PRE"),
# ("BEN", "MD", "PRE"),
# ]

# data_type = 'odr'
# subregion = None
# phase = None
# win_opt_1 = [[1500, 2000], [2000, 2500], [2500, 3000]]
# win_opt_1 = [[x, x + 125] for x in range(1500, 3000, 125)]
# exps = [
#     ('YOUNG', 'PFC'),
#     ('KEN', 'PFC'),
#     ('LEM', 'PFC'),
#     ('ELV', 'PFC'),
#     ('KEN', 'PPC'),
#     ('LEM', 'PPC'),
#     ('ELV', 'PPC'),
# ]

# for exp in exps:
#     data_func = eval(f'load_{data_type}')
#     if data_type.startswith('mnm'):
#         (monkey, subregion, phase) = exp
#         experiment_name = f'{data_type}-{monkey}-{area}-{subregion}-{phase}'
#     elif data_type == 'odr':
#         (monkey, area) = exp
#         experiment_name = f'{data_type}-{monkey}-{area}'
#     elif data_type == 'msng':
#         (monkey, area, subregion) = exp
#         experiment_name = f'{data_type}-{monkey}-{area}-{subregion}'
#     # mlflow.pytorch.autolog()
#     n_trials = 10

#     try_model(experiment_name, data_func, sklearn.neighbors.KNeighborsClassifier(), monkey, area, subregion, phase, [win_opt_1], n_trials, existing_mode='abort')
#     try_model(experiment_name, data_func, sklearn.linear_model.LogisticRegression(max_iter=1000), monkey, area, subregion, phase, [win_opt_1], n_trials, existing_mode='abort')
#     try_model(experiment_name, data_func, sklearn.svm.SVC(), monkey, area, subregion, phase, [win_opt_1], n_trials, existing_mode='abort')
#     try_model(experiment_name, data_func, CrossEntropyFNN, monkey, area, subregion, phase, [win_opt_1], n_trials, existing_mode='abort')

experiment_name = f'{data_type}-single-neuron'
data = prepare_data(eval(f'load_{data_type}'), None, None, None, None, win_opt_1, 120, 1e-5, None)

print(len(data['cells']))

for cell, _ in data['cells'].iterrows():
    try_model(experiment_name, eval(f'load_{data_type}'), CrossEntropyFNN, None, None, None, None, [win_opt_1], 5, existing_mode='abort', cell=cell)

16
Experiment mnm_spatial-single-neuron already has runs with criteria ['tags.cell = "NIN-PFC-AD-PRE-143"', 'tags.model = "CrossEntropyFNN"']
Aborting
Experiment mnm_spatial-single-neuron already has runs with criteria ['tags.cell = "NIN-PFC-AD-PRE-144"', 'tags.model = "CrossEntropyFNN"']
Aborting


[I 2024-10-04 08:53:13,736] A new study created in memory with name: no-name-b00d2492-3fe0-41ea-bbb4-52bf294f5dfb
[I 2024-10-04 08:56:54,179] Trial 0 finished with value: 0.75 and parameters: {'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 7, 'num_hidden_layers': 2}. Best is trial 0 with value: 0.75.


repeat


[I 2024-10-04 09:00:33,445] Trial 1 finished with value: 0.762499988079071 and parameters: {'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 7, 'num_hidden_layers': 2}. Best is trial 1 with value: 0.762499988079071.
[I 2024-10-04 09:04:19,462] Trial 2 finished with value: 0.75 and parameters: {'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375],

{'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 8, 'num_hidden_layers': 3} 0.800000011920929


...

repeat


[I 2024-10-04 12:25:32,113] Trial 3 finished with value: 0.22499999403953552 and parameters: {'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 9, 'num_hidden_layers': 3}. Best is trial 0 with value: 0.22499999403953552.


repeat


[I 2024-10-04 12:29:26,778] Trial 4 finished with value: 0.22499999403953552 and parameters: {'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 9, 'num_hidden_layers': 2}. Best is trial 0 with value: 0.22499999403953552.


{'wins': '[[1500, 1625], [1625, 1750], [1750, 1875], [1875, 2000], [2000, 2125], [2125, 2250], [2250, 2375], [2375, 2500], [2500, 2625], [2625, 2750], [2750, 2875], [2875, 3000], [3000, 3125], [3125, 3250], [3250, 3375], [3375, 3500], [3500, 3625], [3625, 3750], [3750, 3875], [3875, 4000], [4000, 4125], [4125, 4250], [4250, 4375], [4375, 4500], [4500, 4625], [4625, 4750], [4750, 4875], [4875, 5000]]', 'hidden_size_exp': 9, 'num_hidden_layers': 3} 0.22499999403953552


In [36]:
for exp in mlflow.search_experiments():
    if ('msng' in exp.name):
        print(exp.name)
        # mlflow.MlflowClient().delete_experiment(exp.experiment_id)
        # mlflow.MlflowClient().rename_experiment(exp.experiment_id, 'bak-msng-' + exp.name[4:])


msng-single-neuron
msng-LEM-PPC-LIP
msng-KEN-PPC-LIP
msng-LEM-PPC-7a
msng-KEN-PPC-7a
msng-LEM-PPC-None
msng-KEN-PPC-None
msng-LEM-PFC-None
msng-KEN-PFC-None
msng-None-PPC-LIP
msng-None-PPC-7a
msng-None-PFC-8
msng-None-PFC-46
msng-None-PPC-None
msng-None-PFC-None
